<a href="https://colab.research.google.com/github/nsk-ai/RAG-Bootcamp-2025/blob/main/Hybrid_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hybrid Search

In [ ]:
pip install langchain

In [ ]:
pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 8.7 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import os

# Access the secret using userdata.get()
my_variable = userdata.get('GROQ_API_KEY')

# You can also set it as an environment variable for use with os.getenv()
os.environ['GROQ_API_KEY'] = my_variable

In [ ]:
pip install -qU langchain-community chromadb faiss-cpu sentence-transformers rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.2 MB/s eta 0:00

In [ ]:
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_core.documents import Document

# 1. Initialize our Embedding Model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings_model = HuggingFaceEmbeddings(model_name=model_name)

/tmp/ipython-input-1615939869.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Create some dummy documents to highlight the benefits of hybrid search.
# Notice the mix of keyword-heavy and concept-heavy text.
texts = [
    "The quick brown fox jumps over the lazy dog.",
    "A fast, agile fox leaps over a lethargic canine.",
    "The product code for the new widget is WGT-7890. It has an amazing feature set.",
    "Our latest financial report for Q2 2024 shows significant growth in the technology sector.",
    "Financial results for the second quarter of 2024 indicate robust performance.",
    "The concept of quantum entanglement is fascinating, where particles become linked.",
    "Resetting your password is easy. Navigate to the account settings and click 'Forgot Password'.",
    "How do I change my PIN? You can modify your Personal Identification Number in the security section of your profile."
]

In [ ]:
documents = [Document(page_content=t, metadata={"source": f"doc_{i}"}) for i, t in enumerate(texts)]

In [ ]:
# --- 1. Prepare chunks (from Document Loading and Chunking) ---
# In a real app, this would come after loading documents. For this example, our texts are already small.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = text_splitter.split_documents(documents)
print(f"Total chunks prepared: {len(chunks)}")

Total chunks prepared: 8


In [ ]:
# --- 2. Create a Vector Store (ChromaDB) for Dense Search ---
# Using in-memory Chroma for simplicity. In a real project, you'd persist this.
vectorstore = Chroma.from_documents(chunks, embeddings_model)
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
print("\nDense (VectorStore) Retriever created.")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Dense (VectorStore) Retriever created.


In [ ]:
# --- 3. Create a BM25 Retriever for Sparse Search ---
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 5
print("Sparse (BM25) Retriever created.")

Sparse (BM25) Retriever created.


### Example 1: Basic Hybrid Search with Equal Weights

This is the most common starting point. We give equal importance to both sparse and dense search.

In [ ]:
print("\n--- Example 1: Basic Hybrid Search (Equal Weights) ---")
# Initialize the EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.5, 0.5] # 50/50 weighting
)


--- Example 1: Basic Hybrid Search (Equal Weights) ---


In [ ]:
# A query with a strong keyword but also a conceptual part
query_hybrid_1 = "info on product WGT-7890"
print(f"Query: '{query_hybrid_1}'")
retrieved_docs_hybrid_1 = ensemble_retriever.invoke(query_hybrid_1)

Query: 'info on product WGT-7890'


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
print(f"\nRetrieved {len(retrieved_docs_hybrid_1)} documents with Hybrid Search:")
for i, doc in enumerate(retrieved_docs_hybrid_1):
    print(f"  {i+1}. Source: {doc.metadata.get('source', 'N/A')}, Content: '{doc.page_content}'")


Retrieved 6 documents with Hybrid Search:
  1. Source: doc_2, Content: 'The product code for the new widget is WGT-7890. It has an amazing feature set.'
  2. Source: doc_7, Content: 'How do I change my PIN? You can modify your Personal Identification Number in the security section of your profile.'
  3. Source: doc_5, Content: 'The concept of quantum entanglement is fascinating, where particles become linked.'
  4. Source: doc_4, Content: 'Financial results for the second quarter of 2024 indicate robust performance.'
  5. Source: doc_3, Content: 'Our latest financial report for Q2 2024 shows significant growth in the technology sector.'
  6. Source: doc_6, Content: 'Resetting your password is easy. Navigate to the account settings and click 'Forgot Password'.'


In [ ]:
# Now, let's compare this to what each retriever would have found on its own.
# This is crucial for understanding WHY hybrid is better.
print("\n--- Individual Retriever Comparison for the Same Query ---")
print("BM25 Results (strong on keywords):")
bm25_results_1 = bm25_retriever.invoke(query_hybrid_1)
for i, doc in enumerate(bm25_results_1[:3]): # Show top 3
    print(f"  {i+1}. Source: {doc.metadata.get('source', 'N/A')}, Content: '{doc.page_content}'")


--- Individual Retriever Comparison for the Same Query ---
BM25 Results (strong on keywords):
  1. Source: doc_2, Content: 'The product code for the new widget is WGT-7890. It has an amazing feature set.'
  2. Source: doc_7, Content: 'How do I change my PIN? You can modify your Personal Identification Number in the security section of your profile.'
  3. Source: doc_5, Content: 'The concept of quantum entanglement is fascinating, where particles become linked.'


In [ ]:
print("\nVectorStore Results (might miss the specific code):")
vector_results_1 = vector_retriever.invoke(query_hybrid_1)
for i, doc in enumerate(vector_results_1[:3]): # Show top 3
    print(f"  {i+1}. Source: {doc.metadata.get('source', 'N/A')}, Content: '{doc.page_content}'")


VectorStore Results (might miss the specific code):
  1. Source: doc_2, Content: 'The product code for the new widget is WGT-7890. It has an amazing feature set.'
  2. Source: doc_3, Content: 'Our latest financial report for Q2 2024 shows significant growth in the technology sector.'
  3. Source: doc_4, Content: 'Financial results for the second quarter of 2024 indicate robust performance.'


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


### Example 2: Hybrid Search with Weighted Preference

What if we know our users primarily ask conceptual questions, but we still want keyword search as a safety net? We can adjust the weights.


In [ ]:
print("\n--- Example 2: Hybrid Search (Preferring Dense Search) ---")
# Let's say we generally prefer semantic results, but still want keyword backup.
ensemble_retriever_biased = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.3, 0.7] # 30% BM25, 70% VectorStore
)


--- Example 2: Hybrid Search (Preferring Dense Search) ---


In [ ]:
# A query that is strongly semantic
query_hybrid_2 = "how can I change my personal identification number?"
print(f"Query: '{query_hybrid_2}'")
retrieved_docs_hybrid_2 = ensemble_retriever_biased.invoke(query_hybrid_2)

print(f"\nRetrieved {len(retrieved_docs_hybrid_2)} documents with Biased Hybrid Search:")
for i, doc in enumerate(retrieved_docs_hybrid_2):
    print(f"  {i+1}. Source: {doc.metadata.get('source', 'N/A')}, Content: '{doc.page_content}'")

Query: 'how can I change my personal identification number?'

Retrieved 6 documents with Biased Hybrid Search:
  1. Source: doc_7, Content: 'How do I change my PIN? You can modify your Personal Identification Number in the security section of your profile.'
  2. Source: doc_6, Content: 'Resetting your password is easy. Navigate to the account settings and click 'Forgot Password'.'
  3. Source: doc_5, Content: 'The concept of quantum entanglement is fascinating, where particles become linked.'
  4. Source: doc_3, Content: 'Our latest financial report for Q2 2024 shows significant growth in the technology sector.'
  5. Source: doc_2, Content: 'The product code for the new widget is WGT-7890. It has an amazing feature set.'
  6. Source: doc_4, Content: 'Financial results for the second quarter of 2024 indicate robust performance.'


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
# Example with a numerical filter (year must be 2024)
# Note: ChromaDB supports operators like $eq, $ne, $gt, $gte, $lt, $lte
recent_report_retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2, "filter": {"year": {"$eq": 2024}}}
)

In [ ]:
query_numerical = "What are the latest developments?"
retrieved_num_filtered = recent_report_retriever.invoke(query_numerical)
print(f"\n--- Filtered Dense Search for: '{query_numerical}' (Year = 2024) ---")
for i, doc in enumerate(retrieved_num_filtered):
    print(f"Doc {i+1}: Content='{doc.page_content}'")
    print(f"   Source='{doc.metadata.get('source')}', Year='{doc.metadata.get('year')}'\n")


--- Filtered Dense Search for: 'What are the latest developments?' (Year = 2024) ---
Doc 1: Content='Our quarterly report shows a 15% growth in the tech sector, largely driven by AI advancements.'
   Source='financial-report', Year='2024'

Doc 2: Content='The new CEO, Sarah Jenkins, announced a company-wide focus on AI integration.'
   Source='press-release', Year='2024'



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
